![alt text](https://zewailcity.edu.eg/main/images/logo3.png)

_Prepared by_  [**Muhammad Hamdy AlAref**](mailto:malaref@zewailcity.edu.eg)

# Informed Search

Unlike uninformed search algorithms, informed algorithms try to take advantage of problem-specific knowledge to solve the problem more effectively. Such information can be encapsulated with what we will call a *heuristic* function. A heuristic function is an evaluation function that *estimates* the number of steps from a given state to the nearest goal state.

## Problem Formulation

Problem formulation, as mentioned before, is the first step in solving it! This time with a heuristic function!

In [1]:
class Problem:
    '''
    Abstract base class for problem formulation that supports a heuristic function.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None
    
    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError
    
    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError
    
    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError
    
    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

    def heuristic(self, state):
        '''Returns the heuristic value of the given state, i.e., the estimated number of steps to the nearest goal state.'''
        raise NotImplementedError

## Node Data Structure

This is the same node structure from the uninformed search with some extra fields for informed search algorithms. Again, just a class for some required bookkeeping!

In [2]:
class Node:
    '''Node data structure for search space bookkeeping.'''
    
    def __init__(self, state, parent, action, path_cost, heuristic):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.g = path_cost
        self.h = heuristic
        self.f = path_cost + heuristic

    @classmethod
    def root(cls, problem):
        '''Factory method to create the root node.'''
        init_state = problem.init_state
        return cls(init_state, None, None, 0, problem.heuristic(init_state))

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        child_state = problem.result(parent.state, action)
        return cls(
            child_state,
            parent,
            action,
            parent.g + problem.step_cost(parent.state, action),
            problem.heuristic(child_state))

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.g
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

## Example Algorithm: Greedy Best-First Search

Greedy best-first search is an informed algorithm that uses the heuristic value to choose the next node to expand. It is... well, *greedy* so it is *not optimal*! It follows the most *promising* node and that's it!

In [3]:
from heapq import heappush, heappop
from itertools import count

counter = count()

def greedy_best_first(problem, verbose=False):
    '''Greedy best-first search implementation.'''
    frontier = [(None, None, Node.root(problem))]
    explored = set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        _, _, node = heappop(frontier)
        if node.state in explored: continue
        if problem.goal_test(node.state):
            return solution(node)
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                heappush(frontier, (child.h, next(counter), child))

## Visualizer

Again, this is the same `Visualizer` we used before to help visualize the algorithm's progress.

In [4]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for _, _, node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

## Example: Sliding Puzzle

We will be using the same toy problem, [sliding puzzle](https://en.wikipedia.org/wiki/Sliding_puzzle), to see the effect of adding a heuristic function on the number of states expanded!

In [5]:
class SlidingPuzzle3x3(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}
    
    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves
    
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        expected_cost = 0
        for x, y in zip(state, self._goal_state):
            if x != y: expected_cost += 1
        return expected_cost

def _sliding_puzzle_3x3_visualizer(problem, state):
    '''Custom visualizer for the 3x3 sliding puzzle problem.'''
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

_visualizers[SlidingPuzzle3x3] = _sliding_puzzle_3x3_visualizer

Let's try solving the sliding puzzle with greedy best first search!

In [6]:
problem = SlidingPuzzle3x3('12345678 ', '123 56478')

In [7]:
greedy_best_first(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
--------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 
--------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
   5 6 
 4 7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 
--------------------------------------------------------------------------------


(['left', 'left', 'up'], 3)

## Comparison with BFS

Now, let's compare the performance of greedy best-first search against the BFS uninformed algorithm from the previous time!

In [44]:
from collections import deque

def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([(None, None, Node.root(problem))])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        _, _, node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.appendleft((None, None, child))
                explored.add(child.state)

In [45]:
%prun print(bfs_graph(SlidingPuzzle3x3('183756 42', '12345678 ')))

(['right', 'right', 'up', 'left', 'up', 'left', 'down', 'right', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'right', 'right', 'down'], 20)
 

In [46]:
%prun print(greedy_best_first(SlidingPuzzle3x3('183756 42', '12345678 ')))

(['up', 'right', 'up', 'right', 'down', 'down', 'left', 'up', 'right', 'up', 'left', 'down', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down'], 24)
 

## Requirement

### Part A

Let's resolve the sliding puzzle problem with other informed search algorithms!

You are required to write Python code that implements the following algorithms,

1. Uniform-Cost Search (optimal uninformed)
2. A* Search (optimal informed if the heuristic is admissible/consistent)

You may think this is a lot, but it is not! If you think about it, each algorithm requires only ONE change in the already given greedy best-first search implementation.

After the algorithms are implemented, apply them to the sliding puzzle problem and compare the solutions with the one breadth-first search found! Should the solutions differ? How many states were expanded in each one?

**HINT** IPython's `%prun` magic command can help you compare the algorithms' performance!

### Part B

Add a heuristic function to your formulation of the *2D maze* problem from the previous lab and solve it using uniform-cost search, greedy best-first search and A* search! Should the solultions differ? How many states were expanded in each one?

**HINT** Again, IPython's `%prun` magic command can help you compare the algorithms' performance!

**Estimated time for this exercise is 1 hour!**

# **Solution**

## Part A

### 1. Uniform-Cost Search

In [49]:
def uni_cost_search(problem, verbose=False):
    '''Uniform Cost Search BFS graph search implementation.'''
    frontier = [(None, None, Node.root(problem))]
    explored = set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        _, _, node = heappop(frontier)
        if node.state in explored: continue
        if problem.goal_test(node.state):
            return solution(node)
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                heappush(frontier, (child.g, next(counter), child))

In [51]:
uni_cost_search(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
--------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
--------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
 7   8 

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 4

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 
--------------------------------------------------------------------------------
Frontier at step 5

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1   2 
 4 5 3 
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 6

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1   2 
 4 5 3 
 7 8 6 

 1   3 
 4 2 5 
 7 8 6 

 1 2 3 
 4 8 5 
 7   6 

 1 2 

(['left', 'left', 'up'], 3)

### 2. A* Search

In [52]:
def a_star_search(problem, verbose=False):
    '''Uniform Cost Search BFS graph search implementation.'''
    frontier = [(None, None, Node.root(problem))]
    explored = set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        _, _, node = heappop(frontier)
        if node.state in explored: continue
        if problem.goal_test(node.state):
            return solution(node)
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                heappush(frontier, (child.f, next(counter), child))

In [53]:
a_star_search(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
--------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
   5 6 
 4 7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------


(['left', 'left', 'up'], 3)

## Part B

## _Optional_ Generic Sliding Puzzle

For the sake of completeness, this is the same generalized formulation for the sliding puzzle problem that enables solving sliding puzzles of any size and labels but with the heuristic function implemented. This is completely optional.

**DO NOT WASTE TIME UNDERSTANDING IT TILL YOU ARE DONE WITH THE REQUIREMENT!**

In [ ]:
class SlidingPuzzle(Problem):
    '''Generic Sliding Puzzle problem formulation with arbitrary width and height.'''
    
    blank = ' '

    def __init__(self, init_state, goal_state, width=None, height=None):
        def parse(state):
            '''Auxiliary function for parsing the input into a tuple.'''
            blanks = set(char for char in state if not char.isalnum())
            if len(blanks) is 1:
                blank = blanks.pop()
            elif len(blanks) is 2:
                sep = blanks.pop()
                blank = blanks.pop()
                if state.count(blank) > state.count(sep): sep, blank = blank, sep
                state = state.split(sep)
            else:
                raise ValueError
            state = tuple(state)
            index = state.index(blank)
            return state[:index] + (SlidingPuzzle.blank,) + state[index + 1:]
        self.init_state = parse(init_state)
        self._goal_state = parse(goal_state)
        state_len = len(self.init_state)
        assert state_len is len(self._goal_state)
        self._width = width if width else state_len // height if height else round(state_len ** 0.5)
        self._height = height if height else state_len // width if width else round(state_len ** 0.5)
        assert self._width * self._height is state_len
        self._action_values = {'up': -self._width, 'down': +self._width, 'left': -1, 'right': +1}
    
    def actions(self, state):
        index = state.index(SlidingPuzzle.blank)
        if index // self._width > 0:
            yield 'up'
        if index // self._width < self._height - 1:
            yield 'down'
        if index % self._width > 0:
            yield 'left'
        if index % self._width < self._width - 1:
            yield 'right'
    
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(SlidingPuzzle.blank)
        return swap(state, index, index + self._action_values[action])
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(x != y for x, y in zip(state, self._goal_state))

def _sliding_puzzle_visualizer(problem, state):
    '''Custom visualizer for the sliding puzzle problem.'''
    element_width = max(map(len, state))
    for i in range(0, problem._width * problem._height, problem._width):
        print([i.center(element_width) for i in (state[i:i + problem._width])])

_visualizers[SlidingPuzzle] = _sliding_puzzle_visualizer

In [ ]:
greedy_best_first(SlidingPuzzle('1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 #', '1 2 3 4 5 6 7 8 9 10 11 12 # 13 14 15'), verbose=True)

Frontier at step 1

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '15', '  ']
--------------------------------------------------------------------------------
Frontier at step 2

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '  ', '15']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '  ']
['13', '14', '15', '12']
--------------------------------------------------------------------------------
Frontier at step 3

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '  ', '14', '15']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '  ']
['13', '14', '15', '12']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '  ', '12']
['13', '14', '11', '15']
--------------------------------------------------------------------------------
Frontier at step 4

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']

(['left', 'left', 'left'], 3)